In [13]:
from pdf2df import convert_to_df
# df = convert_to_df('../multipdf/RITTAL Invoice 4220025684.PDF')
df = convert_to_df("../pdf_data/Rittal/Rittal - 4220014716 - CP267.pdf")
df

{'format': 'PDF 1.6', 'title': '', 'author': 'IE1JOBS ', 'subject': '', 'keywords': '', 'creator': 'Form ZSD_INVOICE_RITTAL_TG EN', 'producer': 'SAP NetWeaver 740 ', 'creationDate': 'D:20200408060043Z', 'modDate': "D:20200430130242+01'00'", 'trapped': '', 'encryption': None}


Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no
0,Rittal,45,113,65,124,0,0,1
1,Limited,67,113,95,124,0,0,1
2,Barrowside,98,113,138,124,0,0,1
3,Business,140,113,172,124,0,0,1
4,"Park,",175,113,193,124,0,0,1
...,...,...,...,...,...,...,...,...
330,please,324,215,341,223,13,0,2
331,state,343,215,356,223,13,0,2
332,the,358,215,366,223,13,0,2
333,above-mentioned,368,215,414,223,13,0,2


In [14]:
def find_text(temp_df, text,case=False):
    location = temp_df[temp_df['TEXT'].str.contains(text,case=case, na=False)].index.tolist()
    return location

def find_next_word(temp_df, text, nearest_text, case=False):
    pos = find_text(temp_df,text,case=case)
    pos = [item +1 for item in pos]
#     print(pos)
    new_pos = list()
    for item in pos:
#         temp = df.loc[]
        temp_value= temp_df.loc[item]['TEXT']
#         print(temp_value.lower())
       
        if temp_value.lower().__contains__(nearest_text.lower()):
            new_pos.append(item-1)
        
    return new_pos

In [15]:

df = df[df['page_no']==1].reset_index(drop=True)
df = df.sort_values(['Y1','X1']).reset_index(drop=True)

# df

In [16]:
# pos= find_text(df, 'Invoice', 'No.')
# invoice_no = df[pos[1]-1:pos[1]]['TEXT'].values[0]
# print(invoice_no)

In [17]:
pos = find_text(df, 'Invoice')
df[pos[1]:pos[1]+10]
invoice_no = df[pos[1]+3:pos[1]+4]['TEXT'].values[0]
invoice_no

'4220014716'

In [18]:
pos = find_text(df,'date')
invoice_date = df[pos[0]+2:pos[0]+3]['TEXT'].values[0]
print(invoice_date)

07.04.2020


In [19]:
df['page_no'].unique().tolist()

[1]

In [20]:
table_end = find_text(df, 'Item',case =True)
table_end

[107]

In [27]:
table_start = find_text(df,'Quantity')
print(table_start[1])
table_end = find_text(df, 'Item',case =True)
if table_end.__len__() < 2:
    table_end = find_text(df,'items', case=True)
print(table_end)
table_data = df[table_start[1]:table_end[-1]]
table_data

116
[152]


,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no
116,Quantity,263,400,304,414,13,3,1
117,unit,306,400,325,414,13,3,1
118,in,522,400,531,414,15,0,1
119,EUR,534,400,555,414,15,0,1
120,1,45,427,50,440,16,0,1
121,1073500,75,427,114,440,16,1,1
122,1,272,427,277,440,16,2,1
123,EA,283,427,296,440,16,2,1
124,"141,86",391,427,422,440,16,3,1
125,EUR,428,427,449,440,16,3,1


In [24]:
find_text(df,'PU',case=True)
pos = find_text(df, '27')
df[pos[0]:pos[0]+4]

IndexError: list index out of range

In [28]:
from pprint import pprint
msg = dict()
result = list()
for index, row in table_data.iterrows():
    
    if (40 <row['X1'] < 52):
#         print(row['TEXT'])
        if ('part' in msg):
            result.append(msg)
            pprint(msg)
            
        msg =dict()
        msg['item'] = row['TEXT']
        msg['part']= table_data.loc[index+1]['TEXT']
        msg['desc'] = ''
        
    
    if row['TEXT'].__contains__('disc'):
        msg['disc'] = table_data.loc[index+1]['TEXT']
    
    if('part' in msg) and (row['TEXT'].__contains__('EA') or row['TEXT'].__contains__('PU')) and ('qty' not in msg):
        msg['qty'] = table_data.loc[index-1]['TEXT']
    
    elif ('part' in msg) and (70 < row['X1'] < 230):
        msg['desc']= msg['desc']+" "+row['TEXT']
           
    elif ('qty' in msg) and (385  < row['X1'] < 422) and ('unit' not in msg):
        msg['unit'] = row['TEXT']
        
    elif ('unit' in msg) and (510 < row['X1'] < 565) and ('total' not in msg):
        msg['total'] = row['TEXT']

    
result.append(msg)
        
        

{'desc': ' 1073500 AE1073 CABINET 760x760x300 RAL7035 WHD: 760 x 760 x 300 mm',
 'item': '1',
 'part': '1073500',
 'qty': '1',
 'total': '141,86',
 'unit': '141,86'}
{'desc': ' 1015600 AE1015 S/S CABINET 400X500X210 WHD 400x500x210',
 'item': '2',
 'part': '1015600',
 'qty': '1',
 'total': '195,95',
 'unit': '195,95'}


KeyError: 152

In [29]:
result

[{'item': '1',
  'part': '1073500',
  'desc': ' 1073500 AE1073 CABINET 760x760x300 RAL7035 WHD: 760 x 760 x 300 mm',
  'qty': '1',
  'unit': '141,86',
  'total': '141,86'},
 {'item': '2',
  'part': '1015600',
  'desc': ' 1015600 AE1015 S/S CABINET 400X500X210 WHD 400x500x210',
  'qty': '1',
  'unit': '195,95',
  'total': '195,95'}]